## Making the dadi input file from Stacks VCF output

In [13]:
### Note that this function NOW works for vcf files containing many pops!

import vcf
import re

def vcf_to_dadi(VCF_file, outgrp_list, out_file):
    
    VCF_handle = open(VCF_file, 'r') ## make handle
    
    my_vcf = vcf.Reader(VCF_handle) ## read in data with vcf module
    
    ### Get population names (from first snp in file), and write header line to input file ###
    
    snp = my_vcf.next()
    names = []
    for sample in snp.samples:
        names.append(sample.sample.split("_")[0].strip("0123456789-"))
    names = sorted(set(names))
    print "Populations (n="+str(len(names))+"):" 
    print ', '.join(names)
    header_line = "In_REF\tOut_REF\tAllele1\t"+'\t'.join(names)+"\tAllele2\t"+'\t'.join(names)+"\tTag_ID\tSite"
    
    dadi_input = open(out_file, 'w')
    
    dadi_input.write(header_line+"\n") ## write header line
    
    
    ##### Now get the data ######
    
    VCF_handle = open(VCF_file, 'r') ## make handle
    my_vcf = vcf.Reader(VCF_handle) ## Parse file again, as .next() will have moved the iterator on
    
    ## For each SNP in the file ...
    
    for snp in my_vcf:

        GT_dict = {}
        
        ## get outgroup ref allele and data
        
        out_ref_count = 0
        out_alt_count = 0
        
        for sample in snp.samples:
            if any([sample.sample.startswith(a) for a in outgrp_list]):
                if sample['GT'] == "0/0":
                    out_ref_count += 2
                elif sample['GT'] == "0/1":
                    out_ref_count += 1
                    out_alt_count += 1  ## counting one allele twice
                elif sample['GT'] == "1/0":
                    out_ref_count += 1
                    out_alt_count += 1
                elif sample['GT'] == "1/1":
                    out_alt_count += 2
        
        if out_ref_count <= out_alt_count:
            OUT_REF_ALLELE = snp.ALT[0]
            GT_dict["Outgroup"]=(out_alt_count,out_ref_count)
        elif out_alt_count < out_ref_count:
            OUT_REF_ALLELE = snp.REF
            GT_dict["Outgroup"]=(out_ref_count,out_alt_count) 
        
        
        #### for all non outgroup populations ####


        for population in names:
            in_ref_count = 0
            in_alt_count = 0
                        
            GT_dict[population] = ()

            for sample in snp.samples:
                
                if sample.sample.startswith (population) and not any([sample.sample.startswith(a) for a in outgrp_list]):
                    
                    if sample['GT'] == "0/0":
                        in_ref_count += 2
                    elif sample['GT'] == "0/1":
                        in_ref_count += 1
                        in_alt_count += 1
                    elif sample['GT'] == "1/0":
                        in_ref_count += 1
                        in_alt_count += 1
                    elif sample['GT'] == "1/1":
                        in_alt_count += 2                            
            print population, in_ref_count, in_alt_count
            if in_alt_count <= in_ref_count:
                IN_REF_ALLELE = snp.REF
                IN_ALT_ALLELE = snp.ALT[0]
                GT_dict[population]=(in_ref_count,in_alt_count)                   
            elif in_ref_count < in_alt_count:
                IN_REF_ALLELE = snp.ALT[0]
                IN_ALT_ALLELE = snp.REF
                GT_dict[population]=(in_alt_count, in_ref_count)  
                            
        in_REF_counts =[]
        in_ALT_counts =[]
        
        for population in names:
            in_REF_counts.append(str(GT_dict[population][0]))
            in_ALT_counts.append(str(GT_dict[population][1]))


        snp_line = "-"+str(IN_REF_ALLELE)+"-"+"\t"+"-"+str(OUT_REF_ALLELE)+"-"+"\t"+str(IN_REF_ALLELE)+"\t"+str('\t'.join(in_REF_counts))+"\t"+str(IN_ALT_ALLELE)+"\t"+str("\t".join(in_ALT_counts))+"\t"+str(snp.ID)+"\t"+str(snp.POS)
        dadi_input.write(snp_line+"\n")

In [15]:
vcf_to_dadi("/media/dan/34D5D1CE642D7E36/2013076_Hanfling_Bernd/dadi/TESTER.vcf", ["RM31", "RM32"], "/media/dan/34D5D1CE642D7E36/2013076_Hanfling_Bernd/dadi/TEST.vcf")

Populations (n=35):
BF, BOR, CA-LK, CAKE, COP, DEND, EP, FM, GBP, GF, H2K, H2KH, HK, HOLT, LAS, MOAT, MY, OBY, OU, PED, POLEN, PRO, RM, SD, SK, STEC, STYV, SWED, SWED-H, SWEDG, TROM, TU, V, VIKKHY, WEN
BF 18 0
BOR 10 0
CA-LK 20 0
CAKE 18 0
COP 20 0
DEND 10 0
EP 4 0
FM 12 0
GBP 10 0
GF 10 0
H2K 10 0
H2KH 4 0
HK 6 0
HOLT 30 0
LAS 6 0
MOAT 20 0
MY 10 0
OBY 18 0
OU 16 0
PED 16 0
POLEN 20 0
PRO 18 0
RM 22 0
SD 0 0
SK 20 0
STEC 20 0
STYV 18 0
SWED 8 0
SWED-H 2 0
SWEDG 2 0
TROM 16 0
TU 18 0
V 23 1
VIKKHY 12 0
WEN 18 0
BF 18 0
BOR 10 0
CA-LK 20 0
CAKE 18 0
COP 20 0
DEND 10 0
EP 4 0
FM 12 0
GBP 10 0
GF 10 0
H2K 10 0
H2KH 4 0
HK 6 0
HOLT 28 2
LAS 6 0
MOAT 20 0
MY 10 0
OBY 18 0
OU 16 0
PED 16 0
POLEN 20 0
PRO 18 0
RM 22 0
SD 18 0
SK 20 0
STEC 20 0
STYV 18 0
SWED 8 0
SWED-H 2 0
SWEDG 2 0
TROM 18 0
TU 18 0
V 12 0
VIKKHY 12 0
WEN 18 0
BF 18 0
BOR 10 0
CA-LK 10 10
CAKE 18 0
COP 20 0
DEND 10 0
EP 0 0
FM 12 0
GBP 10 0
GF 6 0
H2K 10 0
H2KH 4 0
HK 6 0
HOLT 26 2
LAS 6 0
MOAT 20 0
MY 10 0
OBY 9 9
OU 10 4
P